In [21]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_preview.csv')

# Certifique-se de que não há NaNs nos dados
df['Preco_Real'].fillna(method='ffill', inplace=True)

# Supõe-se que haja uma coluna de data em 'df' que represente as datas correspondentes a 'Preco_Real'
df['Data'] = pd.to_datetime(df['Data'])

# Definindo 'last_date' com a última data do DataFrame
last_date = df['Data'].iloc[-1]

# Escalar os dados para o intervalo [0, 1]
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df['Preco_Real'].values.reshape(-1, 1))

# Função para criar um conjunto de dados
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
    return np.array(Xs)

# Configurações da série temporal
time_steps = 30  # Use 30 dias para criar a sequência
X = create_dataset(df_scaled, time_steps)
y = df_scaled[time_steps:]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Alterar o número de unidades LSTM
model = Sequential()
model.add(LSTM(units=70, return_sequences=True, input_shape=(X_train.shape[1], 1)))  # Mais unidades
model.add(LSTM(units=70))  # Mais unidades
model.add(Dense(units=1))

# Criar um otimizador com uma taxa de aprendizagem específica
optimizer = Adam(learning_rate=0.0001)

# Compilar o modelo com o otimizador configurado
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Adicionando Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Treinar o modelo com early stopping e um batch size diferente
model.fit(X_train, y_train, epochs=100, batch_size=64, callbacks=[early_stopping], validation_split=0.1)

# Prever os próximos 7 dias
# Começar com a última sequência disponível
current_sequence = X[-1].reshape((1, time_steps, 1))
predicted_next_7_days = []

for _ in range(7):
    # Fazer previsão utilizando a sequência atual
    next_predicted = model.predict(current_sequence)
    # Adicionar a previsão à lista
    predicted_next_7_days.append(next_predicted[0, 0])
    # Atualizar a sequência para incluir a nova previsão
    next_predicted = next_predicted.reshape((1, 1, 1))
    current_sequence = np.append(current_sequence[:, 1:, :], next_predicted, axis=1)

# Escalar de volta as previsões para os valores originais
predicted_next_7_days = scaler.inverse_transform(np.array(predicted_next_7_days).reshape(-1, 1))

# Criar as datas para os próximos 7 dias
next_7_days = [last_date + pd.Timedelta(days=x) for x in range(1, 8)]

# Associar as previsões com as datas
predictions_with_dates = [(date.date(), value[0]) for date, value in zip(next_7_days, predicted_next_7_days)]

# Printar as previsões com as datas correspondentes
for date, value in predictions_with_dates:
    print(f'Data: {date}, Valor previsto: {value:.2f}')


Epoch 1/100
48/48 [==============================] - 4s 33ms/step - loss: 0.0332 - val_loss: 0.0038
Epoch 2/100
48/48 [==============================] - 1s 21ms/step - loss: 0.0018 - val_loss: 8.9420e-04
Epoch 3/100
48/48 [==============================] - 1s 21ms/step - loss: 6.5353e-04 - val_loss: 7.0918e-04
Epoch 4/100
48/48 [==============================] - 1s 21ms/step - loss: 5.9171e-04 - val_loss: 6.6576e-04
Epoch 5/100
48/48 [==============================] - 1s 21ms/step - loss: 5.4865e-04 - val_loss: 6.3307e-04
Epoch 6/100
48/48 [==============================] - 1s 21ms/step - loss: 5.1776e-04 - val_loss: 6.0601e-04
Epoch 7/100
48/48 [==============================] - 1s 23ms/step - loss: 5.0435e-04 - val_loss: 5.8922e-04
Epoch 8/100
48/48 [==============================] - 1s 21ms/step - loss: 4.7353e-04 - val_loss: 5.6440e-04
Epoch 9/100
48/48 [==============================] - 1s 21ms/step - loss: 4.5777e-04 - val_loss: 5.5419e-04
Epoch 10/100
48/48 [====================

In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Carregar os dados
df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/weather_preview.csv')

# Certifique-se de que não há NaNs nos dados
df['Preco_Real'].fillna(method='ffill', inplace=True)

# Supõe-se que haja uma coluna de data em 'df' que represente as datas correspondentes a 'Preco_Real'
df['Data'] = pd.to_datetime(df['Data'])

# Definindo 'last_date' com a última data do DataFrame
last_date = df['Data'].iloc[-1]

# Escalar os dados para o intervalo [0, 1]
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df['Preco_Real'].values.reshape(-1, 1))

# Função para criar um conjunto de dados
def create_dataset(X, time_steps=1):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps)]
        Xs.append(v)
    return np.array(Xs)

# Configurações da série temporal
time_steps = 10  # Use 30 dias para criar a sequência
X = create_dataset(df_scaled, time_steps)
y = df_scaled[time_steps:]

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Criar o modelo LSTM
model = Sequential()
model.add(LSTM(units=60, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=60))
model.add(Dense(units=1))

# Criar um otimizador com uma taxa de aprendizagem específica
optimizer = Adam(learning_rate=0.0001)

# Compilar o modelo com o otimizador configurado
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Adicionando Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Treinar o modelo com early stopping e um batch size diferente
model.fit(X_train, y_train, epochs=100, batch_size=50, callbacks=[early_stopping], validation_split=0.1)

# Prever os próximos dias úteis
current_sequence = X[-1].reshape((1, time_steps, 1))
predicted_next_days = []

current_date = last_date

for _ in range(7):
    for _ in range(7):
        current_date += pd.Timedelta(days=1)
        next_predicted = model.predict(current_sequence)
        predicted_next_days.append((current_date, next_predicted[0, 0]))
        next_predicted = next_predicted.reshape((1, 1, 1))
        current_sequence = np.append(current_sequence[:, 1:, :], next_predicted, axis=1)


# Escalar de volta as previsões para os valores originais
predicted_values = np.array([value for _, value in predicted_next_days if value is not None])
predicted_values_scaled_back = scaler.inverse_transform(predicted_values.reshape(-1, 1))

# Associar as previsões com as datas
predictions_with_dates = []
for (current_date, _), value in zip(predicted_next_days, predicted_values_scaled_back):
    predictions_with_dates.append((current_date.date(), value[0] if value is not None else None))

# Printar as previsões com as datas correspondentes
for date, value in predictions_with_dates:
    if value is not None:
        print(f'Data: {date}, Valor previsto: {value:.2f}')
    else:
        print(f'Data: {date}, Previsão não realizada para fins de semana.')


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/weather_preview.csv'

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Carregar os dados de treino que incluem preços e dados climáticos
df_train = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_preview.csv')
df_train.fillna(method='ffill', inplace=True)
df_train['Data'] = pd.to_datetime(df_train['Data'])

# Selecionar as colunas de dados climáticos e de preço
climate_features = ['PT1', 'PA1', 'TPO1', 'TMA1', 'TME1', 'TMI1', 'URA1', 'URAMI1', 'VRMA1', 'VVME1', 'PT2', 'PA2', 'TPO2', 'TMA2', 'TME2', 'TMI2', 'URA2', 'URAMI2', 'VRMA2', 'VVME2', 'PT4', 'PA4', 'TPO4', 'TMA4', 'TME4', 'TMI4', 'URA4', 'URAMI4', 'VRMA4', 'VVME4', 'PT6', 'PA6', 'TPO6', 'TMA6', 'TME6', 'TMI6', 'URA6', 'URAMI6', 'VRMA6', 'VVME6']  # Substitua pelos nomes reais das características climáticas
target_column = 'Preco_Real'
features = climate_features + [target_column]

# Escalar os dados
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

df_train_scaled = df_train.copy()
df_train_scaled[features] = scaler_features.fit_transform(df_train[features])

# Função para criar um conjunto de dados que inclui os dados climáticos e o preço
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:(i + time_steps), :]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 30  # Use 30 dias para criar a sequência

# Preparar o conjunto de dados de entrada e saída
X, y = create_dataset(df_train_scaled[climate_features].values, df_train_scaled[target_column].values, time_steps)

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Modelo
model = Sequential()
model.add(LSTM(units=70, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=70, return_sequences=False))
model.add(Dense(units=1))

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')

# Adicionar Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=64, callbacks=[early_stopping], validation_split=0.1)

# Carregar os dados de resposta que contêm apenas informações climáticas
df_response = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/respostas.csv')
df_response['Data'] = pd.to_datetime(df_response['Data'])
df_response_scaled = scaler_features.transform(df_response[climate_features])

# Criar o conjunto de dados de entrada para previsão
X_response = []
for i in range(len(df_response_scaled) - time_steps):
    X_response.append(df_response_scaled[i:(i + time_steps), :-1])
X_response = np.array(X_response)

# Prever os próximos valores
predicted_prices_scaled = model.predict(X_response)
predicted_prices = scaler_features.inverse_transform(np.hstack((predicted_prices_scaled, np.zeros((predicted_prices_scaled.shape[0], len(climate_features) - 1)))))

# Exibir as previsões
predicted_prices = predicted_prices[:, 0]  # Selecionar apenas a coluna de preços
for i, price in enumerate(predicted_prices):
    print(f'Previsão {i+1}: {price:.2f}')


Epoch 1/100
47/47 [==============================] - 4s 38ms/step - loss: 0.0521 - val_loss: 0.0362
Epoch 2/100
47/47 [==============================] - 1s 29ms/step - loss: 0.0439 - val_loss: 0.0367
Epoch 3/100
47/47 [==============================] - 1s 29ms/step - loss: 0.0412 - val_loss: 0.0320
Epoch 4/100
47/47 [==============================] - 1s 28ms/step - loss: 0.0369 - val_loss: 0.0273
Epoch 5/100
47/47 [==============================] - 1s 28ms/step - loss: 0.0334 - val_loss: 0.0291
Epoch 6/100
47/47 [==============================] - 1s 28ms/step - loss: 0.0305 - val_loss: 0.0243
Epoch 7/100
47/47 [==============================] - 1s 28ms/step - loss: 0.0288 - val_loss: 0.0248
Epoch 8/100
47/47 [==============================] - 1s 29ms/step - loss: 0.0260 - val_loss: 0.0220
Epoch 9/100
47/47 [==============================] - 1s 28ms/step - loss: 0.0249 - val_loss: 0.0236
Epoch 10/100
47/47 [==============================] - 1s 29ms/step - loss: 0.0240 - val_loss: 0.0207

KeyboardInterrupt: 

In [28]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import joblib  # Updated import statement

# Assuming that 'climate_features' and 'your_model.pkl' paths are correct
climate_features = ['Data', 'PT1', 'PA1', 'TPO1', 'TMA1', 'TME1', 'TMI1', 'URA1', 'URAMI1', 'VRMA1', 'VVME1', 'PT2', 'PA2', 'TPO2', 'TMA2', 'TME2', 'TMI2', 'URA2', 'URAMI2', 'VRMA2', 'VVME2', 'PT4', 'PA4', 'TPO4', 'TMA4', 'TME4', 'TMI4', 'URA4', 'URAMI4', 'VRMA4', 'VVME4', 'PT6', 'PA6', 'TPO6', 'TMA6', 'TME6', 'TMI6', 'URA6', 'URAMI6', 'VRMA6', 'VVME6']

# Load initial dataset to fit the scaler
df_initial = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_preview.csv')
df_initial['Data'] = pd.to_datetime(df_initial['Data'])

# Initialize the MinMaxScaler
scaler_features = MinMaxScaler()

# Fit the scaler on your initial dataset without 'Preco_Real'
scaler_features.fit(df_initial[climate_features])

# Load new data to be used for predictions
df_response = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/respostas.csv')
df_response['Data'] = pd.to_datetime(df_response['Data'])

# Scale the new data with the fitted scaler
df_response_scaled = scaler_features.transform(df_response[climate_features])

# Load your trained model
model = joblib.load('your_model.pkl')  # Replace with your model's filename

# Make predictions
predictions = model.predict(df_response_scaled)

# Add predictions to the DataFrame
df_response['Preco_Real_Predicted'] = predictions

# Output the DataFrame with the predictions
df_response.to_csv('predictions.csv', index=False)  # Save the predictions to a CSV file

print("Predictions have been saved to 'predictions.csv'.")


TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'n

In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Substitua 'caminho_para_seu_arquivo.csv' pelo caminho do seu arquivo CSV
precos_df = pd.read_csv('C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_preview.csv')

# Combina o DataFrame de preços com o DataFrame de temperaturas baseando-se na coluna 'Data'
# Isso pressupõe que ambos os DataFrames têm uma coluna 'Data' que pode ser usada para unir os dados
df['Data'] = pd.to_datetime(df['Data'])
precos_df['Data'] = pd.to_datetime(precos_df['Data'])
df = pd.merge(df, precos_df, on='Data', how='inner')

# Calcula a média diária das temperaturas mínimas
df['TMI_media'] = df[['TMI1', 'TMI2', 'TMI4', 'TMI6']].mean(axis=1)

# Vamos supor que você tenha uma coluna 'preco_real' em seu dataframe
# df['preco_real'] = [...seus dados de preço real...]

# Agora, calculamos a correlação
correlacao = df[['TMI_media', 'Preco_Real']].corr()

# Imprime a matriz de correlação
print(correlacao)

# Para visualização, vamos plotar um gráfico de dispersão
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='TMI_media', y='preco_real')
plt.title('Correlação entre Temperatura Média Mínima e Preço Real do Café')
plt.xlabel('Temperatura Média Mínima (°C)')
plt.ylabel('Preço Real do Café')
plt.show()


            TMI_media  Preco_Real
TMI_media    1.000000    0.012706
Preco_Real   0.012706    1.000000


ValueError: Could not interpret value `preco_real` for `y`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>